In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
gpu_id = '0'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
dataset_name = 'trentino'
sub_set_name = 'call'
series_type = 'hourly'

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def preprocess(data, std=True):
    '''
    Function to do preprocess for input data
    inputs:
        + data: pandas.DataFrame
    outputs:
        + preprocessed_data: pandas.DataFrame
    '''
    if std:
        scaler = StandardScaler()
        temp = scaler.fit_transform(data)
    else:
        scaler = MinMaxScaler()
        temp = scaler.fit_transform(data)
    processed_data = pd.DataFrame(temp, index=data.index, columns = data.columns)
    # processed_data = processed_data.to_numpy()
    return processed_data

In [6]:
file_path = f"./dataset/{dataset_name}/{dataset_name}_{sub_set_name}_{series_type}.csv"
raw_data = pd.read_csv(file_path)
raw_data = raw_data.pivot(index='startTime', columns='gridID', values=sub_set_name)
raw_data = raw_data.fillna(0)
raw_data = preprocess(raw_data, std=False)
raw_data.head()

gridID,38,39,40,154,155,156,157,158,159,160,...,11217,11218,11219,11220,11335,11336,11337,11452,11453,11454
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.026968,0.027979,0.054033,0.022686,0.025051,0.021806,0.017165,0.063940,0.182172,0.064736,...,0.002259,0.000069,0.001301,0.002970,0.004161,0.000727,0.002413,0.003107,0.003388,0.003513
2013-11-01 01:00:00,0.002192,0.009714,0.078341,0.008246,0.010590,0.018950,0.024315,0.031946,0.144847,0.040950,...,0.003489,0.000139,0.000000,0.000000,0.005900,0.001271,0.003543,0.005616,0.005598,0.004459
2013-11-01 02:00:00,0.027031,0.013580,0.067715,0.004549,0.008183,0.020699,0.027771,0.051970,0.197436,0.009968,...,0.002573,0.000084,0.000661,0.000147,0.005642,0.000365,0.000447,0.004947,0.001383,0.000377
2013-11-01 03:00:00,0.004477,0.005250,0.019870,0.000000,0.000000,0.005578,0.003652,0.010927,0.015063,0.019474,...,0.000125,0.000000,0.000000,0.000795,0.000378,0.000000,0.000000,0.000000,0.000019,0.000000
2013-11-01 04:00:00,0.005706,0.006306,0.021477,0.004549,0.005886,0.008242,0.010755,0.010265,0.026710,0.016291,...,0.000260,0.000069,0.000245,0.001927,0.000299,0.000298,0.012260,0.000233,0.004342,0.011156


In [7]:
raw_data.shape

(1488, 6259)

In [8]:
# prompt: write pandas to csv

import pandas as pd

# Save the DataFrame to a CSV file
raw_data.rename(columns={'startTime': 'date'}, inplace=True)
raw_data.to_csv(f"./dataset/{dataset_name}/{sub_set_name}.csv", index_label='date')
raw_data.head()

gridID,38,39,40,154,155,156,157,158,159,160,...,11217,11218,11219,11220,11335,11336,11337,11452,11453,11454
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.026968,0.027979,0.054033,0.022686,0.025051,0.021806,0.017165,0.063940,0.182172,0.064736,...,0.002259,0.000069,0.001301,0.002970,0.004161,0.000727,0.002413,0.003107,0.003388,0.003513
2013-11-01 01:00:00,0.002192,0.009714,0.078341,0.008246,0.010590,0.018950,0.024315,0.031946,0.144847,0.040950,...,0.003489,0.000139,0.000000,0.000000,0.005900,0.001271,0.003543,0.005616,0.005598,0.004459
2013-11-01 02:00:00,0.027031,0.013580,0.067715,0.004549,0.008183,0.020699,0.027771,0.051970,0.197436,0.009968,...,0.002573,0.000084,0.000661,0.000147,0.005642,0.000365,0.000447,0.004947,0.001383,0.000377
2013-11-01 03:00:00,0.004477,0.005250,0.019870,0.000000,0.000000,0.005578,0.003652,0.010927,0.015063,0.019474,...,0.000125,0.000000,0.000000,0.000795,0.000378,0.000000,0.000000,0.000000,0.000019,0.000000
2013-11-01 04:00:00,0.005706,0.006306,0.021477,0.004549,0.005886,0.008242,0.010755,0.010265,0.026710,0.016291,...,0.000260,0.000069,0.000245,0.001927,0.000299,0.000298,0.012260,0.000233,0.004342,0.011156


In [9]:
# Download the dataset
# download_elec_dataset()

# # Clean dataset
# elec_data = clean_elec()

# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Std_normalization = 1
# if Std_normalization:
#     scaler = StandardScaler()
#     temp = scaler.fit_transform(elec_data)
#     norm_means = scaler.mean_
#     norm_std = scaler.scale_
# else:
#     scaler = MinMaxScaler()
#     temp = scaler.fit_transform(elec_data)

# elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

In [10]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(raw_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
# args.seq_len = 144
args.root_path='./dataset/trentino/'
args.data_path='call.csv'
args.data_path

'call.csv'

In [11]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [12]:
test_data[0][0].shape

(144, 6259)

In [13]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [14]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(625, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(625, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
        

# Setup WANDB

In [15]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.2
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Milan dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Trentino Call dataset",
    # Hyperparams
    config={
        "dataset": "Milan Call",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 10000,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [16]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 cost time: 3.4667294025421143
Epoch: 2 cost time: 2.6527645587921143
Epoch: 3 cost time: 2.6981542110443115
Epoch: 4 cost time: 2.865831136703491
Epoch: 5 cost time: 2.490647315979004
Epoch: 1 cost time: 2.8498897552490234
Epoch: 2 cost time: 3.0386555194854736
Epoch: 3 cost time: 2.88816499710083
Epoch: 4 cost time: 2.934234857559204
Epoch: 5 cost time: 2.966726064682007


Progress:   2%|▏         | 1/50 [00:38<31:03, 38.03s/it]

Epoch: 1 cost time: 2.7554004192352295
Epoch: 2 cost time: 2.5068986415863037
Epoch: 3 cost time: 2.6579344272613525
Epoch: 4 cost time: 2.787174940109253
Epoch: 5 cost time: 2.98048996925354
Epoch: 1 cost time: 2.9016096591949463
Epoch: 2 cost time: 2.6196467876434326
Epoch: 3 cost time: 2.660902976989746
Epoch: 4 cost time: 2.5969889163970947
Epoch: 5 cost time: 2.7893178462982178


Progress:   4%|▍         | 2/50 [01:14<29:36, 37.01s/it]

...

Progress:  96%|█████████▌| 48/50 [29:38<01:14, 37.40s/it]

Epoch: 1 cost time: 2.442469596862793
Epoch: 2 cost time: 2.516339063644409
Epoch: 3 cost time: 2.4856932163238525
Epoch: 4 cost time: 2.6754398345947266
Epoch: 5 cost time: 2.5375661849975586
Epoch: 1 cost time: 2.7586638927459717
Epoch: 2 cost time: 2.7202510833740234
Epoch: 3 cost time: 2.975102663040161
Epoch: 4 cost time: 2.5372016429901123
Epoch: 5 cost time: 2.6203906536102295


Progress:  98%|█████████▊| 49/50 [30:13<00:36, 36.68s/it]

Epoch: 1 cost time: 3.133073091506958
Epoch: 2 cost time: 2.902512788772583
Epoch: 3 cost time: 2.8903017044067383
Epoch: 4 cost time: 2.834585428237915
Epoch: 5 cost time: 2.739711284637451
Epoch: 1 cost time: 2.616636276245117
Epoch: 2 cost time: 2.921295642852783
Epoch: 3 cost time: 3.081422805786133
Epoch: 4 cost time: 2.9454855918884277
Epoch: 5 cost time: 2.934123992919922


Progress: 100%|██████████| 50/50 [30:51<00:00, 37.02s/it]
